In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from astropy.io import fits, ascii
from astropy import wcs
from astropy.coordinates import SkyCoord
from astropy import units as u

from IPython.display import Image
from skimage.io import imread

from toasty import toast, cartesian_sampler, normalizer

from collections import defaultdict, namedtuple

import ps1skycell_toast as pssc

In [2]:
def minmax(arr):
    return [min(arr),max(arr)]

In [ ]:
def M101Sampler():
    
    def vec2Pix(raArr,decArr):
        
        tile = np.zeros(raArr.shape)
        
        raArr[raArr < 0] += 2 * np.pi # making all ras 0 - 2pi
        
        raCorners = np.array([raArr[0][0],raArr[0][-1],raArr[-1][0],raArr[-1][-1]])
        decCorners = np.array([decArr[0][0],decArr[0][-1],decArr[-1][0],decArr[-1][-1]])       
        minRa = min(raCorners)
        maxRa = max(raCorners)
        minDec = min(decCorners)
        maxDec = max(decCorners)

        
        #   THIS IS ALL TEMPORARY 
        m101Pos = SkyCoord(210.79995, 54.34410, frame='icrs', unit='deg')
        if ((m101Pos.ra.rad >= minRa) and (m101Pos.ra.rad <= maxRa)):
            if ((m101Pos.dec.rad >= minDec) and (m101Pos.dec.rad <= maxDec)):
   
                fitsFile = fits.open('rings.v3.skycell.2381.053.stk.r.unconv.fits')
                rfWcs = wcs.WCS(fitsFile[1].header)
                footprint=np.array([SkyCoord(*x,frame='icrs',unit='deg') for x in rfWcs.calc_footprint(fitsFile[1].header) ])
                imgData = fitsFile[1].data
                fitsFile.close()
                
                fpMinRa = min(x.ra.rad for x in footprint)#(min(footprint[:,0])*u.deg).to(u.rad)
                fpMaxRa = max(x.ra.rad for x in footprint)#(max(footprint[:,0])*u.deg).to(u.rad)
                fpMinDec = min(x.dec.rad for x in footprint)#(min(footprint[:,1])*u.deg).to(u.rad)
                fpMaxDec = max(x.dec.rad for x in footprint)#(max(footprint[:,1])*u.deg).to(u.rad)

                print("woot")
                fpMask = (raArr <= fpMaxRa)  & (raArr >= fpMinRa) & (decArr <= fpMaxDec) & (decArr >= fpMinDec)
                print(np.sum(fpMask))
                print(fpMinRa,fpMaxRa)
                print(fpMinDec,fpMaxDec)
                print()
                
                ny, nx = imgData.shape[0:2]    
                
                tile[fpMask] = imgData[(ny*(decArr - fpMinDec)/(fpMaxDec - fpMinDec)).astype(int)[fpMask],
                                       (nx*(1-(raArr - fpMinRa)/(fpMaxRa - fpMinRa))).astype(int)[fpMask]]
                                
        # ^ THIS IS ALL TEMPORARY ^

        
        
        return tile
    
    return vec2Pix

In [ ]:
sampler = normalizer(M101Sampler(),-3,10)
output = 'toasts/ps1_M101'
depth = 8  
toast(sampler, depth, output, ra_range=[205,215],dec_range=[50,60])

In [ ]:
sampler = 'toasts/ps1_bottom'
output = 'toasts/ps1_bottom'
depth = 4 
toast(sampler, depth, output)

In [ ]:
def panstarrsSampler():
    
    psCells = ascii.read("filter_r_rings.rpt")
    psSC2FileLoc = np.full((max(psCells['SCn']) + 1,max(psCells['SCm']) + 1),"",dtype='<U168')
    #psSC2FileLoc[psCells['SCn'],psCells['SCm']] = psCells['fileNPath']
    psSC2FileLoc[2404][5]  = "RINGS.V3.skycell.2404.005.stk.4032678.unconv.fits"
    psSC2FileLoc[2381][62] = "RINGS.V3.skycell.2381.062.stk.3906396.unconv.fits"
    psSC2FileLoc[2381][63] = "RINGS.V3.skycell.2381.063.stk.3906397.unconv.fits"
    psSC2FileLoc[2381][64] = "RINGS.V3.skycell.2381.064.stk.3906398.unconv.fits"
    psSC2FileLoc[2381][52] = "RINGS.V3.skycell.2381.052.stk.3906388.unconv.fits"
    psSC2FileLoc[2381][53] = "rings.v3.skycell.2381.053.stk.r.unconv.fits" # M101
    psSC2FileLoc[2381][54] = "RINGS.V3.skycell.2381.054.stk.3906390.unconv.fits"
    psSC2FileLoc[2381][42] = "RINGS.V3.skycell.2381.042.stk.3906380.unconv.fits"
    psSC2FileLoc[2381][43] = "RINGS.V3.skycell.2381.043.stk.3906381.unconv.fits"
    psSC2FileLoc[2381][44] = "RINGS.V3.skycell.2381.044.stk.3906382.unconv.fits"

    def vec2Pix(raArr,decArr):
        
        tile = np.zeros(raArr.shape) # this should be filled with whatever we want to signal "no data" 
                                     # I am currently using zero
        tileFill = np.full(raArr.shape,False,bool)

        #print(raArr[0,0], decArr[0,0])
        raArr[raArr < 0] += 2 * np.pi # making all ras 0 - 2pi
        
        minRa,maxRa = minmax(np.array([raArr[0][0],raArr[0][-1],raArr[-1][0],raArr[-1][-1]]))
        minDec,maxDec = minmax(np.array([decArr[0][0],decArr[0][-1],decArr[-1][0],decArr[-1][-1]]))

        while(False in tileFill):
            inds = np.where(tileFill == False) # is there a better way to do this?
            
            # These 2 lines will need to be generalized away eventually
            pc,sc = pssc.findskycell(np.rad2deg(raArr[inds[0][0],inds[1][0]]), 
                                     np.rad2deg(decArr[inds[0][0],inds[1][0]]))
            dataFle = psSC2FileLoc[pc[0],sc[0]]
            #print(raArr[inds[0][0],inds[1][0]], decArr[inds[0][0],inds[1][0]])
            #print(np.rad2deg(raArr[inds[0][0],inds[1][0]]), 
            #      np.rad2deg(decArr[inds[0][0],inds[1][0]]))
            
            if not dataFle:
                tileFill[inds[0][0],inds[1][0]] = True
                # Do I want to/ can I deal with the full footprint here?
                continue
            
            print("Skycell:",pc,",",sc)
            print("File location:",dataFle)
            
            # getting what we need out of the fits file
            fitsFile = fits.open(dataFle)
            dfWcs = wcs.WCS(fitsFile[1].header)
            fp = dfWcs.calc_footprint(fitsFile[1].header)
            footprint = np.array([SkyCoord(*x,frame='icrs',unit='deg') for x in  fp])
            #print(fp)
            #print(footprint)
            imgData = fitsFile[1].data
            fitsFile.close()
            
            # getting the edges, not the following code assumes the footprint is a rectangle aligned
            # alont ra/dec lines, this might need to be altered for the general case 
            # (galex can be the experiment for this) (might need to use the mask in the fits file?)
            imMinRa,imMaxRa = minmax([x.ra.rad for x in footprint])
            imMinDec,imMaxDec = minmax([x.dec.rad for x in footprint])

            # There are problems if the ra interval crosses the circle boundary (e.g. 359 - 1)
            # This should not happen with dec
            # Check and account for that here
            if (footprint[0].ra - footprint[3].ra) > 0:  
                # this is what would need to be altered to take into account different footprint shapes
                imMask = (raArr <= imMaxRa)  & (raArr >= imMinRa) 
                imMask &= (decArr <= imMaxDec) & (decArr >= imMinDec)
            else: # Crosses circle boundary
                imMask = (raArr >= imMaxRa)  & (raArr <= 2*np.pi)
                imMask |= ((raArr >= 0)  & (raArr <= imMinRa))
                imMask &= (decArr <= imMaxDec) & (decArr >= imMinDec)
        
            ny, nx = imgData.shape[0:2]    
            tile[imMask] = imgData[(ny*(decArr - imMinDec)/(imMaxDec - imMinDec)).astype(int)[imMask],
                                   (nx*(1-(raArr - imMinRa)/(imMaxRa - imMinRa))).astype(int)[imMask]]
            print("Number of pixels to be filled:",np.sum(imMask))
            tileFill |= imMask # adding true everywhere we just filled
            print()
                                
        return tile
    
    return vec2Pix

In [ ]:
sampler = normalizer(panstarrsSampler(),-3,10)
output = 'toasts/ps1_M101'
depth = 8  
toast(sampler, depth, output, ra_range=[210.3,211],dec_range=[54,55.6])